### Deployment Notebook

This notebook is designed to test different deployment approaches and includes a **Gradio interface**.

- **Usage**:
  - There is an option `USE_LLM=false` that allows the interface to run normally without downloading the LLM.
  - If you wish to set `USE_LLM=true`, the fine-tuned **Gemma 2B** LLM will be downloaded. 
  
  **Note**: This may take time, so it's recommended to use a **GPU** for better performance.


In [1]:
# load the models from huggingface
import pandas as pd
import re
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import torch
import gradio as gr


In [55]:
USE_LLM = False

In [ ]:
regression_model_name = 'wasabibish/plagiarism-detection'
tokenizer_name = 'distilbert/distilbert-base-uncased-finetuned-sst-2-english'
regression_model = AutoModelForSequenceClassification.from_pretrained(regression_model_name)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
similarity_model = SentenceTransformer(tokenizer_name)

In [ ]:
if USE_LLM:
    llm_id = "google/gemma-2b-it"
    llm_id_finetuned = "wasabibish/gemma-2b-it-code-ai-generated"
    llm_tokenizer = AutoTokenizer.from_pretrained(llm_id)
    finetuned_llm = AutoModelForCausalLM.from_pretrained(llm_id)

In [3]:
data = pd.read_csv('data.csv')

In [ ]:
# load the vector db
vector_db = torch.load('vector_db.pt')

## Helper functions

In [41]:
def calculate_weighted_score(sentence1_vector, vector_db,  top_n=3):

    """
    Calculate the weighted score of a given sentence based on the similarity scores 
    of the top_n most similar sentences in the vector_db

    Parameters:
    ----------
    sentence1_vector: np.array
        The vector of the sentence for which the weighted score should be calculated
    vector_db: dict
        A dictionary containing the vectors and scores of the test dataset
    top_n: int
        The number of most similar sentences to consider

    Returns:
    -------
    weighted_score: float
        The weighted score of the sentence
    """

    similarity_scores = {}

    # calculate similarity scores for each sentence in the vector_db
    for sentence, data in vector_db.items():
        vector = data['vector']
        score = data['score']
        similarity = util.pytorch_cos_sim(sentence1_vector, vector).item()
        similarity_scores[sentence] = {
            'similarity': similarity,
            'score': score
        }

    # sort sentences based on similarity scores
    sorted_similarity_scores = sorted(similarity_scores.items(), key=lambda x: x[1]['similarity'], reverse=True)
    most_similar_sentences = sorted_similarity_scores[:top_n]

    # calculate the weighted score (similarity * real score)
    total_score = 0
    total_similarity = 0
    for sentence, data in most_similar_sentences:
        similarity = data['similarity']
        score = data['score']
        total_score += similarity * score
        total_similarity += similarity

    weighted_score = total_score / total_similarity if total_similarity > 0 else 0

    return weighted_score

In [42]:
def get_embedding(sentence, model):
    """
    Get the embedding of a sentence using a given model

    Parameters:
    ----------
    sentence: str
        The sentence to encode
    model: SentenceTransformer
        The model to use for encoding

    Returns:
    -------
    embedding: np.array
        The embedding of the sentence
    """

    embedding = model.encode(sentence)

    return embedding

In [43]:
def get_score_simialrity(question, answer, top_n=3):
    """
    Get the similarity score of a given sentence

    Parameters:
    ----------
    sentence1: str
        The sentence to get the similarity score for

    Returns:
    -------
    similarity_score: float
        The similarity score of the sentence
    """
    text = 'Question: ' + question + '\nAnswer: ' + answer
    sentence1_embedding = get_embedding(text, similarity_model)
    similarity_score = calculate_weighted_score(sentence1_embedding, vector_db, top_n=top_n)

    return similarity_score

In [12]:
def inferece_regression(question, answer=None):
    """"
    Function to make predictions on new data

    Parameters:
    -----------
    question : str
        The question text (coding problem)
    answer : str
        The answer text (solution, given code)

    Returns:
    --------
    float
        The predicted plagiarism score
    """
    if answer is None:
        text = question
    else:
        text = 'Question :\n' + question + '\nAnswer :\n' + answer
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        logits = regression_model(**inputs).logits
    return logits.item()

In [25]:
def get_completion(question, answer):
    """
    Generate a completion for the given question and answer.

    Parameters:
    -----------
    question : str
        The question to be asked. (coding problem)
    answer : str
        The answer to the question. (code snippet)
    model : transformers.PreTrainedModel
        The model to be used for generation.
    tokenizer : transformers.PreTrainedTokenizer
        The tokenizer to be used for tokenization.

    Returns:
    --------
    str
        The generated completion.
    """

    # Define prompt template
    prompt_template = """
    <start_of_turn>user :
      Is this code AI-generated? Provide a score between 0 and 1, where 0 means not AI-generated and 1 means fully AI-generated.

      Question: {question}
      Answer: {answer}

    <end_of_turn>\n<start_of_turn>model :
    """

    # format prompt with query
    prompt = prompt_template.format(question=question, answer=answer)

    # tokenize prompt
    encodeds = llm_tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    # Move inputs to device - ensure model inputs are on the same device as the model
    model_inputs = encodeds.to(finetuned_llm.device)

    # Generate response
    generated_ids = finetuned_llm.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

    # decode response to human readable text
    decoded = llm_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return decoded

In [26]:
def post_process_output(output):
    # Extract the score using regular expressions
    pattern = r"(\d+\.?\d)"  # Matches one or more digits followed by an optional decimal point and more digits
    # get all the matches
    matches = re.findall(pattern, output)

    # Check if any matches were found
    if not matches:
        return None
    # Convert the matches to floats
    scores = [float(match) for match in matches]

    return scores[0]

In [27]:
def get_plagiarism_score(question, answer):
    # Generate completion
    result = get_completion(question=question, answer=answer)

    # Post-process the output
    score = post_process_output(result)

    return score

In [31]:
def plagiarism_detection(question, answer, gemma_llm=False):

    similarity_score = get_score_simialrity(question, answer, 2)
    regression_score = inferece_regression(question, answer)
    if gemma_llm:
        llm_score = get_plagiarism_score(question, answer)
        return llm_score, similarity_score, regression_score
    else:
        return None, similarity_score, regression_score

## Gradio Interface

In [ ]:
gr.Interface(
    fn=plagiarism_detection,
    inputs=[
        gr.Textbox(label="Question", interactive=True),
        gr.Textbox(label="Answer", interactive=True),
        gr.Checkbox(label="Use Gemma LLM")],
    outputs=[gr.Textbox(label="LLM Score"), gr.Textbox(label="Similarity Score"), gr.Textbox(label="Regression Score")],
    title="Plagiarism Detection",
    description="Detect plagiarism in code using a combination of regression and similarity scores.",
    examples=[
        [data['question'][55], data['human_content'][55]],
        [data['question'][75], data['llm_content'][75]]
    ]
).launch()